In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import accuracy_score

## Preparing data

In [105]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.4,
        zoom_range=0.4,
        horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('train', target_size=(150, 150), batch_size=batch_size, class_mode='binary', shuffle=False)
test_generator = test_datagen.flow_from_directory('test', target_size=(150, 150), batch_size=batch_size, class_mode='binary', shuffle=False)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [48]:
train_labels = []

for image, label in train_generator:
    if len(train_labels) == 498:
        break
    train_labels += list(label)
    
test_labels = []

for image, label in test_generator:
    if len(test_labels) == 500:
        break
    test_labels += list(label)

In [49]:
len(train_labels), len(test_labels)

(498, 500)

# Modelling

## Simple model

In [26]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [27]:
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=8
)

Epoch 1/8
100/100 [==============================] - 66s 661ms/step - loss: 0.7902 - accuracy: 0.5558
Epoch 2/8
100/100 [==============================] - 65s 653ms/step - loss: 0.6924 - accuracy: 0.5873
Epoch 3/8
100/100 [==============================] - 65s 651ms/step - loss: 0.7035 - accuracy: 0.5834
Epoch 4/8
100/100 [==============================] - 64s 637ms/step - loss: 0.6438 - accuracy: 0.6463
Epoch 5/8
100/100 [==============================] - 64s 639ms/step - loss: 0.6273 - accuracy: 0.6697
Epoch 6/8
100/100 [==============================] - 64s 636ms/step - loss: 0.6121 - accuracy: 0.6900
Epoch 7/8
100/100 [==============================] - 64s 635ms/step - loss: 0.5868 - accuracy: 0.7022
Epoch 8/8
100/100 [==============================] - 62s 618ms/step - loss: 0.5495 - accuracy: 0.7317


In [28]:
predicts = model.predict_generator(test_generator)
accuracy_score(labels, predicts.reshape(-1) >= 0.5)

0.558

## Using pre-trained VGG-16

In [29]:
from tensorflow.keras.applications import VGG16

In [30]:
vgg_model = VGG16(include_top=False, weights='imagenet')

58892288/58889256 [==============================] - 16s 0us/step


In [106]:
vgg_train = list(vgg_model.predict(train_generator))
vgg_test = list(vgg_model.predict(test_generator))

In [107]:
vgg_train_other = np.array([array.reshape(-1) for array in vgg_train])
vgg_test_other = np.array([array.reshape(-1) for array in vgg_test])

In [121]:
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(vgg_train_other, np.array(train_labels), epochs=30, batch_size=batch_size)

Epoch 1/30
498/498 [==============================] - 1s 1ms/sample - loss: 1.2471 - accuracy: 0.5703
Epoch 2/30
498/498 [==============================] - 0s 902us/sample - loss: 0.6245 - accuracy: 0.6767
Epoch 3/30
498/498 [==============================] - 0s 957us/sample - loss: 0.5279 - accuracy: 0.7570
Epoch 4/30
498/498 [==============================] - 0s 932us/sample - loss: 0.4759 - accuracy: 0.7892
Epoch 5/30
498/498 [==============================] - 0s 958us/sample - loss: 0.4351 - accuracy: 0.8052
Epoch 6/30
498/498 [==============================] - 0s 919us/sample - loss: 0.3800 - accuracy: 0.8213
Epoch 7/30
498/498 [==============================] - 0s 907us/sample - loss: 0.3839 - accuracy: 0.8353
Epoch 8/30
498/498 [==============================] - 0s 900us/sample - loss: 0.3252 - accuracy: 0.8394
Epoch 9/30
498/498 [==============================] - 0s 907us/sample - loss: 0.2209 - accuracy: 0.9076
Epoch 10/30
498/498 [==============================] - 1s 1ms/samp

In [122]:
predictions = model.predict(vgg_test_other).reshape(-1)
accuracy_score([0] * 250 + [1] * 250, predictions >= 0.5)

0.766